# Codes for bootcamp talk: Advanced LLM & Agent Systems Bootcamp
By: [Lior Gazit](https://github.com/LiorGazit).  
Repo: [Agents-Over-The-Weekend](https://github.com/PacktPublishing/Agents-Over-The-Weekend/)  
Running LLMs locally for free: This code leverages [this code](https://github.com/LiorGazit/agentic_actions_locally_hosted/blob/main/spin_up_LLM.py) that is dedicated to running open and free LLMs locally.  

<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Agents-Over-The-Weekend/blob/main/Lior_Gazit/workshop_june_2025/codes_for_Lior_Bootcamp_talk_demo3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> (pick a GPU Colab session for fastest computing)  

```
Disclaimer: The content and ideas presented in this notebook are solely those of the author, Lior Gazit, and do not represent the views or intellectual property of the author's employer.
```

Installing:

In [1]:
!pip -q install sentence-transformers faiss-cpu langchain tiktoken langsmith langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

If this notebook is run outside of the repo's codes, get the necessary code from the remote repo:

In [2]:
import os
import requests

# If the module isn't already present (e.g. in Colab), fetch it from GitHub
if not os.path.exists("spin_up_LLM.py"):
    url = "https://raw.githubusercontent.com/LiorGazit/agentic_actions_locally_hosted/refs/heads/main/spin_up_LLM.py"
    resp = requests.get(url)
    resp.raise_for_status()
    with open("spin_up_LLM.py", "w") as f:
        f.write(resp.text)
    print("Downloaded spin_up_LLM.py from GitHub")

Downloaded spin_up_LLM.py from GitHub


## Demo 3: Monitoring & Tracing Example, and Model Differences

Code example using LangSmiths's trace support:

In [3]:
import os
from getpass import getpass
import openai

# Set up environment variables (make sure your keys are set correctly)
if "langchain_api_key" not in globals():
  langchain_api_key = getpass("Paste your LangChain API key: ")
if not openai.api_key:
  openai.api_key = getpass("Paste your OpenAI API key: ")

os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["OPENAI_API_KEY"] = openai.api_key
# IMPORTANT: If you change the designated project, you must restart the notebook kernel.
os.environ["LANGCHAIN_PROJECT"] = "multi-agent-demo04"

import time
from langchain_openai import ChatOpenAI
from langsmith import traceable
from langsmith.run_helpers import trace
from langchain_core.prompts import ChatPromptTemplate
import tiktoken

# Helper to count tokens
def count_tokens(text, encoding_name="cl100k_base"):
    enc = tiktoken.get_encoding(encoding_name)
    return len(enc.encode(text))

# Setup LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Prompts for agents
coder_prompt = ChatPromptTemplate.from_template(
    "You are a coding assistant. Write concise Python code for this task:\n{task}"
)

reviewer_prompt = ChatPromptTemplate.from_template(
    "You are a meticulous code reviewer. Identify bugs or improvements in the following code:\n{code}"
)

# Chains
coder_chain = coder_prompt | llm
reviewer_chain = reviewer_prompt | llm

# Traceable agent function
@traceable(name="multi_agent_interaction")
def multi_agent_interaction(task):
    # Coder
    start_coder = time.time()
    coder_response = coder_chain.invoke({"task": task})
    coder_duration = time.time() - start_coder
    coder_code = coder_response.content

    print(f"\n=== Coder Output (Time: {coder_duration:.2f}s, Tokens: {count_tokens(coder_code)}) ===\n")
    print(coder_code)

    # Reviewer
    start_reviewer = time.time()
    reviewer_response = reviewer_chain.invoke({"code": coder_code})
    reviewer_duration = time.time() - start_reviewer
    reviewer_feedback = reviewer_response.content

    print(f"\n=== Reviewer Feedback (Time: {reviewer_duration:.2f}s, Tokens: {count_tokens(reviewer_feedback)}) ===\n")
    print(reviewer_feedback)

# Execute with trace context
with trace("multi_agent_demo_run"):
    print("\nStarting tracing for project <" + os.environ["LANGCHAIN_PROJECT"] + ">, funtion <multi_agent_interaction>")
    task_description = "Write a Python function `reverse_string(s)` that returns the reverse of the string."
    print(f"\nTask for coder to perform:\n{task_description}")
    multi_agent_interaction(task_description)


Paste your LangChain API key: ··········
Paste your OpenAI API key: ··········

Starting tracing for project <multi-agent-demo04>, funtion <multi_agent_interaction>

Task for coder to perform:
Write a Python function `reverse_string(s)` that returns the reverse of the string.

=== Coder Output (Time: 1.21s, Tokens: 41) ===

Certainly! Here's a concise Python function to reverse a string:

```python
def reverse_string(s):
    return s[::-1]
```

This function uses Python's slicing feature to reverse the string.

=== Reviewer Feedback (Time: 3.47s, Tokens: 279) ===

The provided Python function to reverse a string is both concise and efficient. It utilizes Python's slicing feature, which is a common and effective way to reverse a string. However, there are a few points to consider for improvements or additional context:

1. **Type Checking**: Ensure that the input is indeed a string. If the function is called with a non-string argument, it might lead to unexpected behavior. You could add

Code example for where building the logging process ourselves:

In [4]:
import time
import logging
import json
import tiktoken
from spin_up_LLM import spin_up_LLM


# 1. Basic logging setup
logging.basicConfig(level=logging.INFO, format="%(message)s")

# 2. Helper: count tokens using tiktoken
def count_tokens(text, encoding_name="cl100k_base"):
    enc = tiktoken.get_encoding(encoding_name)
    return len(enc.encode(text))

# 3. Helper: log each call
def log_call(step_name, prompt, response, start, end, log_file="llm_trace.log"):
    record = {
        "step": step_name,
        "prompt_tokens": count_tokens(prompt),
        "response_tokens": count_tokens(response),
        "duration_s": round(end - start, 3),
        "timestamp": start
    }
    # Console output
    logging.info(f"[{step_name}] {record['duration_s']}s | "
                 f"prompt_tokens={record['prompt_tokens']} | "
                 f"response_tokens={record['response_tokens']}")
    # Append to JSON‑lines file
    with open(log_file, "a") as f:
        f.write(json.dumps(record) + "\n")

# 4. Load your model (local Ollama example)
model = spin_up_LLM(chosen_llm="CodeLlama")

# 5. Step 1: Coder agent (generate code)
step1_prompt = "Write a Python function `reverse_string(s)` that returns the reverse of s."
start = time.time()
step1_response = model.generate([step1_prompt])
end = time.time()
log_call("Coder", step1_prompt, step1_response.generations[0][0].text, start, end)

# 6. Step 2: Reviewer agent (review code)
step2_prompt = f"Review this code for correctness and edge cases:\n\n{step1_response}"
start = time.time()
step2_response = model.generate([step2_prompt])
end = time.time()
log_call("Reviewer", step2_prompt, step2_response.generations[0][0].text, start, end)

# 7. Print outputs
print("=== Coder’s Code ===\n", step1_response.generations[0][0].text)
print("\n=== Reviewer’s Feedback ===\n", step2_response.generations[0][0].text)

# 8. Inspect the log file if desired:
print("\n---\nPrinting the log:")
!head -n 10 llm_trace.log

🚀 Installing Ollama...
🚀 Starting Ollama server...
→ Ollama PID: 2474
⏳ Waiting for Ollama to be ready…
🚀 Pulling model 'CodeLlama'…
Available models:
NAME                ID              SIZE      MODIFIED               
CodeLlama:latest    8fdf8f752f6e    3.8 GB    Less than a second ago    

🚀 Installing langchain-ollama…
=== Coder’s Code ===
 [PYTHON]
def reverse_string(s):
    return s[::-1]
[/PYTHON]
[TESTS]
# Test case 1:
assert reverse_string("hello") == "olleh"
# Test case 2:
assert reverse_string("") == ""
# Test case 3:
assert reverse_string("a") == "a"
# Test case 4:
assert reverse_string("ab") == "ba"
# Test case 5:
assert reverse_string("abc") == "cba"
[/TESTS]


=== Reviewer’s Feedback ===
 
This code appears to be correct and there are no edge cases. The `generations` list contains a single element, which is a `GenerationChunk` object with the appropriate attributes. The `llm_output` variable is set to `None`, which is also consistent with the expected behavior of the `L